In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import torchvision
from torchvision import datasets, models, transforms

In [3]:
from pathlib import Path

In [4]:
from session import *
from callbacks import *
from validation import *
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
DATA_PATH = Path('C:/fastai/courses/dl2/data/road_damage_dataset')

In [7]:
MULTICLASS_CSV_PATH = DATA_PATH/'mc.csv'

In [8]:
im_size = 224

In [9]:
train_tfms = TransformList([
        RandomScale(im_size, 1.2),
        RandomCrop(im_size),
        RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_tfms = TransformList([
        Scale(im_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [10]:
batch_size = 4
data = ImageData.from_csv(DATA_PATH, MULTICLASS_CSV_PATH, batch_size, train_tfms, val_tfms)
num_classes = 8

In [11]:
model_ft = models.resnet18(pretrained=True)
model_ft.avgpool = nn.AdaptiveAvgPool2d(1)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, num_classes),
    nn.Sigmoid()
)

In [12]:
len(data['valid']) * batch_size

1404

In [13]:
criterion = nn.BCELoss()
optim_fn = optim.Adam
sess = Session(model_ft, criterion, optim_fn, [*[1e-3] * 9, 1e-2])
sess.load("TrainLayer1")

In [14]:
accuracy = NHotAccuracy(num_classes)
validator = Validator(data['valid'], accuracy)

In [15]:
validator.run(sess)


Validaton Loss: 0.369633 Validation Accuracy: 0.829351
